In [1]:
import os
import pandas as pd

# Sets the path to the parent directory of RR classes
os.chdir("C:\\Users\\rudra\\OneDrive\\Desktop\\university\\Semester 4\\Assignment_3\\RRcourse2024")

# Import data from the O*NET database, at ISCO-08 occupation level.
task_data = pd.read_csv("Data\onet_tasks.csv")

# Read employment data from Eurostat
isco_sheets = [f"ISCO{i}" for i in range(1, 10)]
isco_data = {}

for sheet in isco_sheets:
    isco_data[sheet] = pd.read_excel("Data\\Eurostat_employment_isco.xlsx", sheet_name=sheet)

# List of countries we are focusing on
countries = ["Belgium", "Spain", "Poland"]

# Initialize a dictionary to store the total workers for each country
total_workers = {country: 0 for country in countries}

# Calculate worker totals in each of the chosen countries
for country in countries:
    total_workers[country] = sum(isco_data[sheet][country] for sheet in isco_sheets)

# Print the total workers for each country
for country, total in total_workers.items():
    print(f"Total workers in {country}: {total}")


Total workers in Belgium: 0     4391.5
1     4472.2
2     4428.3
3     4473.3
4     4426.3
5     4442.1
6     4458.5
7     4471.1
8     4430.4
9     4471.1
10    4493.7
11    4441.7
12    4436.3
13    4467.4
14    4473.1
15    4496.7
16    4477.9
17    4440.2
18    4490.8
19    4481.4
20    4458.4
21    4483.3
22    4503.1
23    4601.3
24    4498.3
25    4536.1
26    4578.9
27    4645.0
28    4633.4
29    4614.8
30    4715.5
31    4736.0
32    4683.8
33    4769.3
34    4784.9
35    4758.2
36    4741.1
37    4682.0
38    4741.1
39    4711.0
Name: Belgium, dtype: float64
Total workers in Spain: 0     18169.2
1     18373.4
2     18241.4
3     17896.2
4     17509.0
5     17500.8
6     17414.9
7     17094.4
8     16799.3
9     16932.9
10    17008.7
11    16916.6
12    16729.4
13    17128.6
14    17263.6
15    17338.3
16    17224.4
17    17633.8
18    17811.1
19    17841.3
20    17792.2
21    18055.6
22    18280.0
23    18251.6
24    18181.9
25    18552.9
26    18780.5
27    18722.5
28    18

In [2]:
import pandas as pd

# Function to read and label ISCO data
def read_and_label_isco(sheet_number):
    df = pd.read_excel("Data\\Eurostat_employment_isco.xlsx", sheet_name=f"ISCO{sheet_number}")
    df['ISCO'] = sheet_number
    return df

# Read and label all ISCO data sheets
isco_data = [read_and_label_isco(i) for i in range(1, 10)]

# Merge all ISCO data into one DataFrame
all_data = pd.concat(isco_data, ignore_index=True)

# List of countries we are focusing on
countries = ["Belgium", "Spain", "Poland"]

# Calculate total workers for each country
total_workers = {}
for country in countries:
    total_workers[country] = sum(all_data[all_data['ISCO'] == i][country] for i in range(1, 10))

# Add total workers to all_data DataFrame and calculate shares
for country in countries:
    all_data[f"total_{country}"] = pd.concat([pd.Series(total_workers[country])] * 9, ignore_index=True)
    all_data[f"share_{country}"] = all_data[country] / all_data[f"total_{country}"]

# Print the first few rows of the resulting DataFrame to verify
print(all_data.head())


      TIME European Union - 28 countries (2013-2020)  Belgium  Czechia  \
0  2011-Q1                                   12850.9    279.0    223.5   
1  2011-Q2                                   12834.2    304.7    226.2   
2  2011-Q3                                   12763.1    348.8    227.3   
3  2011-Q4                                   12668.1    347.9    230.3   
4  2012-Q1                                   12197.3    304.0    231.1   

   Denmark  Spain  Italy  Lithuania  Poland  Finland  Sweden  ISCO  \
0     57.6  879.8  896.0      120.0   963.5    137.2   232.6     1   
1     58.2  911.6  897.0      118.9   918.4    135.8   236.0     1   
2    107.5  881.4  840.1      107.0   913.8    123.5   238.5     1   
3     74.1  888.2  764.9      107.8   935.9    106.6   246.1     1   
4     50.6  848.8  762.2      107.5   952.5    102.5   249.8     1   

   total_Belgium  share_Belgium  total_Spain  share_Spain  total_Poland  \
0            NaN            NaN          NaN          NaN  

In [6]:
import pandas as pd
import numpy as np

# Assume task_data and all_data are already loaded as before

# Extract the first digit of the ISCO variable
task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)

# Calculate mean task values at a 1-digit ISCO level, excluding the 'isco08' column
aggdata = task_data.groupby("isco08_1dig").mean().drop(columns=["isco08"])

# Tasks of interest
tasks_of_interest = [
    "t_4A2a4",
    "t_4A2b2",
    "t_4A4a1"
]

# Merge aggregated task data with all_data on the ISCO code
combined = pd.merge(all_data, aggdata, left_on='ISCO', right_on='isco08_1dig', how='left')

# Function to standardize task values for a given country
def standardize_tasks(country):
    for task in tasks_of_interest:
        task_mean = np.average(combined[task], weights=combined[f"share_{country}"])
        task_std = np.sqrt(np.average((combined[task] - task_mean) ** 2, weights=combined[f"share_{country}"]))
        combined[f"{task}_std_{country}"] = (combined[task] - task_mean) / task_std

# Standardize task values for each country
countries = ["Belgium", "Spain", "Poland"]
for country in countries:
    standardize_tasks(country)

# Print the first few rows of the resulting DataFrame to verify
print(combined.head())


      TIME European Union - 28 countries (2013-2020)  Belgium  Czechia  \
0  2011-Q1                                   12850.9    279.0    223.5   
1  2011-Q2                                   12834.2    304.7    226.2   
2  2011-Q3                                   12763.1    348.8    227.3   
3  2011-Q4                                   12668.1    347.9    230.3   
4  2012-Q1                                   12197.3    304.0    231.1   

   Denmark  Spain  Italy  Lithuania  Poland  Finland  ...  t_4A3b5_rev  \
0     57.6  879.8  896.0      120.0   963.5    137.2  ...     4.270759   
1     58.2  911.6  897.0      118.9   918.4    135.8  ...     4.270759   
2    107.5  881.4  840.1      107.0   913.8    123.5  ...     4.270759   
3     74.1  888.2  764.9      107.8   935.9    106.6  ...     4.270759   
4     50.6  848.8  762.2      107.5   952.5    102.5  ...     4.270759   

   t_4A2a4_std_Belgium  t_4A2b2_std_Belgium  t_4A4a1_std_Belgium  \
0                  NaN                  Na

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import gmean

# List of tasks of interest with their corresponding columns in the DataFrame
tasks = {
    "t_4A2a4": "Analyzing Data or Information",
    "t_4A2b2": "Thinking Creatively",
    "t_4A4a1": "Interpreting the Meaning of Information for Others"
}

# List of countries
countries = ["Belgium", "Poland", "Spain"]

# Function to standardize task values for a given country and task
def standardize_task(country, task_col):
    task_mean = np.average(combined[task_col], weights=combined[f"share_{country}"])
    task_sd = np.sqrt(np.average((combined[task_col] - task_mean) ** 2, weights=combined[f"share_{country}"]))
    combined[f"std_{country}_{task_col}"] = (combined[task_col] - task_mean) / task_sd

# Loop over each task and country to standardize the task values
for task_col in tasks.keys():
    for country in countries:
        standardize_task(country, task_col)

# Print the first few rows of the resulting DataFrame to verify
print(combined.head())


      TIME European Union - 28 countries (2013-2020)  Belgium  Czechia  \
0  2011-Q1                                   12850.9    279.0    223.5   
1  2011-Q2                                   12834.2    304.7    226.2   
2  2011-Q3                                   12763.1    348.8    227.3   
3  2011-Q4                                   12668.1    347.9    230.3   
4  2012-Q1                                   12197.3    304.0    231.1   

   Denmark  Spain  Italy  Lithuania  Poland  Finland  ...  t_4A4a1_std_Poland  \
0     57.6  879.8  896.0      120.0   963.5    137.2  ...                 NaN   
1     58.2  911.6  897.0      118.9   918.4    135.8  ...                 NaN   
2    107.5  881.4  840.1      107.0   913.8    123.5  ...                 NaN   
3     74.1  888.2  764.9      107.8   935.9    106.6  ...                 NaN   
4     50.6  848.8  762.2      107.5   952.5    102.5  ...                 NaN   

   std_Belgium_t_4A2a4  std_Poland_t_4A2a4  std_Spain_t_4A2a4  \
0  